In [5]:
import os
import numpy as np
import cv2
import mediapipe as mp
import tensorflow as tf
# from tensorflow import keras
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

In [7]:
DATASET_PATH = os.path.abspath("organized_data")  # Ensure absolute path
print("Dataset Path:", DATASET_PATH)

train_path = os.path.join(DATASET_PATH, "train")
test_path = os.path.join(DATASET_PATH, "test")

Dataset Path: c:\Users\obada\Desktop\HoldWise Dataset\organized_data


In [8]:
def extract_keypoints(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        keypoints = []
        for landmark in results.pose_landmarks.landmark:
            keypoints.append(landmark.x)
            keypoints.append(landmark.y)
            keypoints.append(landmark.z)
        return np.array(keypoints)
    else:
        return None  # If no pose is detected

In [9]:
def load_dataset(dataset_path):
    data, labels = [], []
    class_map = {"good": 0, "bad": 1, "cant_determine": 2}
    
    for class_name, class_label in class_map.items():
        class_dir = os.path.join(dataset_path, class_name)
        
        if not os.path.exists(class_dir):
            print(f"Warning: {class_dir} does not exist. Skipping.")
            continue
        
        for file in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file)
            keypoints = extract_keypoints(file_path)
            if keypoints is not None:
                data.append(keypoints)
                labels.append(class_label)
    
    return np.array(data), np.array(labels)


In [10]:
X_train, y_train = load_dataset(train_path)
X_test, y_test = load_dataset(test_path)

valiadation_split = 0.2

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=valiadation_split, random_state=42)


In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((35322, 99), (35322,), (11038, 99), (11038,))

In [12]:
X_train = np.nan_to_num(X_train)  # Replace NaN with 0
X_test = np.nan_to_num(X_test)

In [13]:
X_train = X_train / np.linalg.norm(X_train, axis=1, keepdims=True)
X_test = X_test / np.linalg.norm(X_test, axis=1, keepdims=True)

In [14]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 classes
])

In [15]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16)

Epoch 1/50


2208/2208 [==============================] - 4s 1ms/step - loss: 0.5935 - accuracy: 0.7346 - val_loss: 3.1117 - val_accuracy: 0.8153
Epoch 2/50
2208/2208 [==============================] - 3s 1ms/step - loss: 0.4621 - accuracy: 0.8104 - val_loss: 4.8798 - val_accuracy: 0.8113
Epoch 3/50
2208/2208 [==============================] - 3s 1ms/step - loss: 0.4084 - accuracy: 0.8325 - val_loss: 3.4744 - val_accuracy: 0.8475
Epoch 4/50
2208/2208 [==============================] - 3s 1ms/step - loss: 0.3738 - accuracy: 0.8478 - val_loss: 3.8969 - val_accuracy: 0.8520
Epoch 5/50
2208/2208 [==============================] - 2s 1ms/step - loss: 0.3519 - accuracy: 0.8580 - val_loss: 3.9613 - val_accuracy: 0.8262
Epoch 6/50
2208/2208 [==============================] - 2s 1ms/step - loss: 0.3275 - accuracy: 0.8664 - val_loss: 3.3433 - val_accuracy: 0.8596
Epoch 7/50
2208/2208 [==============================] - 2s 1ms/step - loss: 0.3132 - accuracy: 0.8723 - val_loss: 4.1588 - val_accurac

In [20]:
# model.evaluate(X_test, y_test)
loss, accuracy = model.evaluate(X_test, y_test)

345/345 [==============================] - 0s 845us/step - loss: 0.1416 - accuracy: 0.9473


In [21]:
loss, accuracy

(0.14158661663532257, 0.9472730755805969)

In [23]:
# Save the Keras model
model.save("pose_classification_model.h5")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("pose_classification_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite and saved.")

c:\Users\obada\Desktop\HoldWise Dataset\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: C:\Users\obada\AppData\Local\Temp\tmp_9gcqszo\assets


INFO:tensorflow:Assets written to: C:\Users\obada\AppData\Local\Temp\tmp_9gcqszo\assets


Model successfully converted to TensorFlow Lite and saved.


In [22]:
!jupyter nbconvert --to script "C:\Users\obada\Desktop\HoldWise Dataset\pose_classifier_v2.ipynb"
!jupyter nbconvert --to html "C:\Users\obada\Desktop\HoldWise Dataset\pose_classifier_v2.ipynb"

C:\Users\obada\AppData\Local\Programs\Python\Python313\Scripts\jupyter-nbconvert.EXE\__main__.py:4: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
[NbConvertApp] Converting notebook C:\Users\obada\Desktop\HoldWise Dataset\pose_classifier_v2.ipynb to script
[NbConvertApp] Writing 3544 bytes to C:\Users\obada\Desktop\HoldWise Dataset\pose_classifier_v2.py
C:\Users\obada\AppData\Local\Programs\Python\Python313\Scripts\jupyter-nbconvert.EXE\__main__.py:4: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a di